# Deteccion de redundancia

Primera iteracion donde quiero aplicar GCN contrastivo

In [1]:
!pip install -q torch torch-geometric


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.1 MB/s eta 0:00:00


In [4]:
import torch
import numpy as np
from torch_geometric.utils import from_networkx
import networkx as nx
import pickle

# cargamos el grafo con los embeddings
graph_path = "/content/drive/MyDrive/GML/proyecto/graph_law_embeddings.pickle"

with open(graph_path, 'rb') as f:
    G = pickle.load(f)

# Crear un subgrafo solo con nodos que tengan embeddings
# algunos no tienen embeddings porque eran texto vacio,
# debo arreglar cosas en los datos pero para partir continuo
valid_nodes = [n for n, d in G.nodes(data=True) if "embedding" in d and len(d["embedding"]) > 0]
G_valid = G.subgraph(valid_nodes).copy()  # copia para evitar problemas

# Convertir embeddings a matriz numpy
emb_matrix = np.array([G_valid.nodes[n]["embedding"] for n in G_valid.nodes()])

# Crear tensor de embeddings
x = torch.tensor(emb_matrix, dtype=torch.float32)

# Convertir el grafo a objeto Data
from torch_geometric.utils import from_networkx
data = from_networkx(G_valid)
data.x = x

print(data)
print("Número de nodos:", data.num_nodes)
print("Número de aristas:", data.num_edges)
print("Dimensión de embeddings:", data.x.shape)




Data(edge_index=[2, 4032], tipo=[15228], numero=[15228], texto=[15228], organismo=[15228], publlic_name=[15228], embedding=[15228, 384], edge_tipo=[4032], num_nodes=15228, x=[15228, 384])
Número de nodos: 15228
Número de aristas: 4032
Dimensión de embeddings: torch.Size([15228, 384])


/usr/local/lib/python3.12/dist-packages/torch_geometric/utils/convert.py:278: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:253.)
  data_dict[key] = torch.as_tensor(value)


# Arquitectura
Usamos una arquitectura simple GCN

In [5]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class ContrastiveGraphAutoEncoder(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)

    def forward(self, x, edge_index):
        h = F.relu(self.conv1(x, edge_index))
        z = self.conv2(h, edge_index)
        return z

    def reconstruct(self, z):
        # reconstrucción de adyacencia
        return torch.sigmoid(torch.matmul(z, z.T))


# Entrenamiento

Basicamente aca viene la parte de CGL, entrenamos en base al

In [6]:
import torch
import torch.nn.functional as F
from torch import nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = ContrastiveGraphAutoEncoder(in_channels=data.x.shape[1], hidden_channels=256).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
tau = 0.1  # temperatura para contraste
lambda_recon = 0.5

# Proyección de embeddings de texto a la dimensión de la GNN
proj_text = nn.Linear(data.x.shape[1], 256).to(device)

def info_nce_loss(z_graph, z_text, tau=0.2):
    z_g = F.normalize(z_graph, dim=1)
    z_t = F.normalize(z_text, dim=1)
    sim = torch.mm(z_g, z_t.T) / tau
    labels = torch.arange(sim.size(0)).to(sim.device)
    return F.cross_entropy(sim, labels)

data = data.to(device)
x_text = data.x  # embeddings originales de texto

for epoch in range(100):
    model.train()
    optimizer.zero_grad()

    #  Forward GNN
    z_graph = model(data.x, data.edge_index)

    # Proyectar embeddings de texto
    z_text = proj_text(x_text)

    #  Contrastiva
    L_contrast = info_nce_loss(z_graph, z_text, tau)

    # Reconstrucción del grafo
    A_pred = model.reconstruct(z_graph)
    A_real = torch.zeros_like(A_pred)
    A_real[data.edge_index[0], data.edge_index[1]] = 1
    L_recon = F.binary_cross_entropy(A_pred, A_real)

    # Loss total
    loss = L_contrast + lambda_recon * L_recon
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f"Epoch {epoch} | Loss: {loss.item():.4f} (contrast {L_contrast.item():.4f} + recon {L_recon.item():.4f})")



Epoch 0 | Loss: 10.0982 (contrast 9.6514 + recon 0.8938)
Epoch 10 | Loss: 8.4689 (contrast 8.1188 + recon 0.7002)
Epoch 20 | Loss: 7.7527 (contrast 7.4034 + recon 0.6986)
Epoch 30 | Loss: 7.2813 (contrast 6.9328 + recon 0.6971)
Epoch 40 | Loss: 6.9515 (contrast 6.6033 + recon 0.6966)
Epoch 50 | Loss: 6.7025 (contrast 6.3545 + recon 0.6960)
Epoch 60 | Loss: 6.5093 (contrast 6.1616 + recon 0.6954)
Epoch 70 | Loss: 6.3597 (contrast 6.0120 + recon 0.6954)
Epoch 80 | Loss: 6.2486 (contrast 5.9012 + recon 0.6949)
Epoch 90 | Loss: 6.1655 (contrast 5.8179 + recon 0.6951)


In [7]:
model.eval()
with torch.no_grad():
    z_graph = model(data.x, data.edge_index)  # embeddings finales
    z_text = proj_text(data.x)  # embeddings de texto proyectados


In [8]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

embeddings = z_graph.cpu().numpy()

# Usamos simil
sim_matrix = cosine_similarity(embeddings)

# umbral arbitrario
threshold = 0.95
redundant_pairs = np.argwhere(sim_matrix > threshold)

# filtrp
redundant_pairs = [(i, j) for i, j in redundant_pairs if i < j]

print(f"Número de pares redundantes detectados: {len(redundant_pairs)}")


Número de pares redundantes detectados: 1958


In [9]:
import networkx as nx

G_redundant = nx.Graph()
G_redundant.add_edges_from([(i,j) for i,j in redundant_pairs])


In [11]:
import pandas as pd
# Filtrar pares con alta similitud y distinto organismo
threshold_redundancy = 0.5
redundant_cross_org = [
    (i, j, sim_matrix[i,j])
    for i, j in np.argwhere(sim_matrix > threshold_redundancy)
    if i < j and data.organismo[i] != data.organismo[j]  # distinto organismo
]

# Convertimos a DataFrame
cross_org_list = []
for src, dst, sim in redundant_cross_org:
    cross_org_list.append({
        "Ley_1": data.numero[src],
        "Nombre_1": data.publlic_name[src],
        "Organismo_1": data.organismo[src],
        "Ley_2": data.numero[dst],
        "Nombre_2": data.publlic_name[dst],
        "Organismo_2": data.organismo[dst],
        "Similitud": sim
    })

df_cross_org_redundancias = pd.DataFrame(cross_org_list)
df_cross_org_redundancias = df_cross_org_redundancias.sort_values(by="Similitud", ascending=False)

# Top 20
df_cross_org_redundancias.head(50)



,Ley_1,Nombre_1,Organismo_1,Ley_2,Nombre_2,Organismo_2,Similitud
133332,7368,Ley 7368,MINISTERIO DE EDUCACION,11479,Ley 11479,MINISTERIO DE HACIENDA,0.998676
10704,19606,Ley 19606,MINISTERIO DEL INTERIOR,19853,Ley 19853,MINISTERIO DE HACIENDA,0.998654
2235,20403,Ley 20403,MINISTERIO DE HACIENDA,19949,Ley 19949,MINISTERIO DE PLANIFICACION,0.991805
70476,2958,Ley 2958,MINISTERIO DE OBRAS,3235,Ley 3235,MINISTERIO DE FERROCARRIL,0.991277
156976,19163,Ley 19163,MINISTERIO DE HACIENDA,19333,Ley 19333,MINISTERIO DE TRABAJO,0.990364
156975,19163,Ley 19163,MINISTERIO DE HACIENDA,19243,Ley 19243,MINISTERIO DE TRABAJO,0.990281
135925,8102,Ley 8102,MINISTERIO DE HACIENDA,11813,Ley 11813,MINISTERIO DEL INTERIOR,0.990208
132751,7314,Ley 7314,MINISTERIO DE HACIENDA,12605,Ley 12605,MINISTERIO DEL INTERIOR,0.988470
132489,7305,Ley 7305,MINISTERIO DE HACIENDA,9997,Ley 9997,MINISTERIO DEL INTERIOR,0.988455
110005,9610,Ley 9610,MINISTERIO DE HACIENDA,11873,Ley 11873,MINISTERIO DE OBRAS,0.988010
